In [10]:
# Manejo de datos
import pandas as pd

# Modelos de Machine Learning
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


# Preprocesamiento
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Oversampling
from imblearn.over_sampling import SMOTE 
from imblearn.combine import SMOTETomek

# Evaluación de modelos
from sklearn.metrics import classification_report, accuracy_score

# Búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV, train_test_split



**Importo los datos de los CSV provenientes de la API de Spotify**

In [11]:
songs = pd.read_csv('./data/spotify.csv')
spotify_blues = pd.read_csv('./data/spotify_blues.csv')
spotify_country = pd.read_csv('./data/spotify_country.csv')
spotify_electronic = pd.read_csv('./data/spotify_electronic.csv')
spotify_hip_hop = pd.read_csv('./data/spotify_hip_hop.csv')
spotify_jazz = pd.read_csv('./data/spotify_jazz.csv')
spotify_pop = pd.read_csv('./data/spotify_pop.csv')
spotify_rap = pd.read_csv('./data/spotify_rap.csv')
spotify_reggae = pd.read_csv('./data/spotify_reggae.csv')
spotify_rnb = pd.read_csv('./data/spotify_rnb.csv')
spotify_rock = pd.read_csv('./data/spotify_rock.csv')
spotify_mix= pd.read_csv('./data/spotify.csv')


In [12]:
songs = pd.concat([songs, spotify_blues, spotify_country, spotify_electronic, 
                       spotify_hip_hop, spotify_jazz, spotify_pop, 
                       spotify_rap, spotify_reggae, spotify_rnb, spotify_rock,
                       spotify_mix], 
                      ignore_index=True)

In [13]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             2842 non-null   object 
 1   Genre             2842 non-null   object 
 2   Artist            2842 non-null   object 
 3   id                2842 non-null   object 
 4   title_spotify     2842 non-null   object 
 5   album             2842 non-null   object 
 6   sp_popularity     2842 non-null   float64
 7   colab             2842 non-null   object 
 8   release_date      2842 non-null   object 
 9   danceability      2842 non-null   float64
 10  energy            2842 non-null   float64
 11  loudness          2842 non-null   float64
 12  speechiness       2842 non-null   float64
 13  acousticness      2842 non-null   float64
 14  instrumentalness  2842 non-null   float64
 15  liveness          2842 non-null   float64
 16  valence           2842 non-null   float64


**DATA CLEANING**

**La columna "Colab" tiene como valores Y o N (yes o no) lo tranformo en valores booleanos**

In [ ]:
songs['release_date'] = pd.to_datetime(songs['release_date'], errors='coerce')
songs['colab'] = songs['colab'].replace({'Y': True, 'N': False})

In [15]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Title             2842 non-null   object        
 1   Genre             2842 non-null   object        
 2   Artist            2842 non-null   object        
 3   id                2842 non-null   object        
 4   title_spotify     2842 non-null   object        
 5   album             2842 non-null   object        
 6   sp_popularity     2842 non-null   float64       
 7   colab             2842 non-null   bool          
 8   release_date      2597 non-null   datetime64[ns]
 9   danceability      2842 non-null   float64       
 10  energy            2842 non-null   float64       
 11  loudness          2842 non-null   float64       
 12  speechiness       2842 non-null   float64       
 13  acousticness      2842 non-null   float64       
 14  instrumentalness  2842 n

**Transformo la columna "Genre" en tipo categoría por eficiencia a la hora de entrenar el modelo**

In [17]:
songs['Genre'] = songs['Genre'].astype('category')

**Definición de features y conjuntos de datos**

In [27]:
# Seleccionar las características numéricas
numerical_features = ['sp_popularity', 'danceability', 'energy','loudness','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence','key','mode','duration', 'tempo']  # Añade más características si es necesario
X = songs[numerical_features]

# El target (género), en formato categórico
y = songs['Genre']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**RANDOM FOREST**

In [28]:
# Crear el modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo Random Forest: {accuracy:.4f}")

Precisión del modelo Random Forest: 0.4886


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Blues       0.47      0.50      0.48        14
     Country       0.83      0.36      0.50        14
  Electronic       0.50      0.36      0.42        11
     Hip Hop       0.12      0.10      0.11        20
        Jazz       0.72      0.62      0.67        21
         Pop       0.22      0.12      0.16        16
         R&B       0.35      0.27      0.31        22
         Rap       0.12      0.11      0.11        19
      Reggae       0.43      0.46      0.44        13
        Rock       0.58      0.29      0.39        24
 alternative       0.04      0.07      0.05        15
       blues       0.69      0.58      0.63        19
   classical       0.94      1.00      0.97        15
     country       0.70      0.82      0.76        17
       disco       0.53      0.71      0.61        14
  electronic       0.55      0.42      0.48        26
        folk       0.57      0.85      0.68        20
     hip hop       0.30    

El modelo tiene dificultades para predecir correctamente muchas clases, probablemente debido a un desbalance de clases y características menos discriminatorias.
Ajustamos el modelo para tratar de corregir el desbalance de clases.

In [29]:
# Crear el modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Entrenar el modelo
rf.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo Random Forest: {accuracy:.4f}")

Precisión del modelo Random Forest: 0.4938


Vemos que ha mejorado un poco, vamos a probar con SMOTE para corregir más el desbalance.

Además de SMOTE estamos usando SMOTEomek para hacer undersampling de las clases mayoritarias 

In [23]:
smote = SMOTE(random_state=42)

# Aplicar SMOTE para equilibrar los datos de entrenamiento
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

smote_tomek = SMOTETomek(random_state=42)

# Aplicar SMOTETomek para equilibrar los datos de entrenamiento
X_train_res, y_train_res = smote_tomek.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos equilibrados por SMOTE
rf.fit(X_train_smote, y_train_smote)

# Hacer predicciones en el conjunto de prueba
y_pred_smote = rf.predict(X_test)

# Evaluar el modelo con SMOTE
accuracy_smote = accuracy_score(y_test, y_pred_smote)
print(f"Precisión del modelo con SMOTE: {accuracy_smote:.4f}")

# Imprimir un informe de clasificación para SMOTE
print("Informe de clasificación con SMOTE:")
print(classification_report(y_test, y_pred_smote))

# Entrenar el modelo con los datos equilibrados por SMOTETomek
rf.fit(X_train_res, y_train_res)

# Hacer predicciones en el conjunto de prueba
y_pred_res = rf.predict(X_test)

# Evaluar el modelo con SMOTETomek
accuracy_res = accuracy_score(y_test, y_pred_res)
print(f"Precisión del modelo con SMOTETomek: {accuracy_res:.4f}")

# Imprimir un informe de clasificación para SMOTETomek
print("Informe de clasificación con SMOTETomek:")
print(classification_report(y_test, y_pred_res))


Precisión del modelo con SMOTE: 0.4991
Informe de clasificación con SMOTE:
              precision    recall  f1-score   support

       Blues       0.42      0.57      0.48        14
     Country       0.55      0.43      0.48        14
  Electronic       0.38      0.45      0.42        11
     Hip Hop       0.16      0.15      0.15        20
        Jazz       0.76      0.62      0.68        21
         Pop       0.12      0.06      0.08        16
         R&B       0.22      0.18      0.20        22
         Rap       0.16      0.16      0.16        19
      Reggae       0.60      0.46      0.52        13
        Rock       0.60      0.38      0.46        24
 alternative       0.00      0.00      0.00        15
       blues       0.61      0.58      0.59        19
   classical       0.94      1.00      0.97        15
     country       0.72      0.76      0.74        17
       disco       0.56      0.71      0.62        14
  electronic       0.52      0.42      0.47        26
      

Ha mejorado un poco, pero sigue siendo poco preciso


**XGBoost**

In [31]:
# Codificar las etiquetas de los géneros (de texto a numérico)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Escalar las características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)

# Aplicar SMOTE para equilibrar las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Crear el modelo de XGBoost
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Entrenar el modelo con los datos equilibrados por SMOTE
xgb_model.fit(X_train_smote, y_train_smote)

# Hacer predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# Decodificar las predicciones para que se correspondan con los nombres originales de los géneros
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test_decoded, y_pred_decoded)
print(f"Precisión del modelo XGBoost con SMOTE: {accuracy:.4f}")

# Imprimir un informe de clasificación
print(classification_report(y_test_decoded, y_pred_decoded))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:46:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Precisión del modelo XGBoost con SMOTE: 0.4584
              precision    recall  f1-score   support

       Blues       0.45      0.36      0.40        25
     Country       0.50      0.32      0.39        25
  Electronic       0.14      0.14      0.14        14
     Hip Hop       0.17      0.16      0.16        25
        Jazz       0.73      0.48      0.58        33
         Pop       0.25      0.15      0.19        26
         R&B       0.30      0.24      0.27        29
         Rap       0.24      0.21      0.23        28
      Reggae       0.41      0.52      0.46        21
        Rock       0.50      0.42      0.46        33
 alternative       0.00      0.00      0.00        23
       blues       0.59      0.53      0.56        36
   classical       0.75      1.00      0.86        21
     country       0.59      0.63      0.61        27
       disco       0.47      0.58      0.52        24
  electronic       0.47      0.38      0.42        39
        folk       0.60      0.78 

Peor que el intento anterior, de momento el que nos ha dado mejores resultados ha sido Random Forest con SMOTE y SMOTETomek.

**Probamos con un grid search**

In [33]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0)  # Cambiar verbose a 0

# Ajustar el modelo con los datos equilibrados por SMOTE
grid_search.fit(X_train_smote, y_train_smote)

# Usar el mejor modelo
best_rf = grid_search.best_estimator_

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

# Hacer predicciones en el conjunto de prueba
y_pred_best = best_rf.predict(X_test)

# Evaluar el modelo optimizado
accuracy_best = accuracy_score(y_test, y_pred_best)

# Imprimir resultados finales
print(f"Mejores parámetros: {best_params}")  # Agregar impresión de los mejores parámetros
print(f"Precisión del modelo Random Forest optimizado: {accuracy_best:.4f}")
print(classification_report(y_test, y_pred_best))

Mejores parámetros: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Precisión del modelo Random Forest optimizado: 0.4596
              precision    recall  f1-score   support

           0       0.35      0.44      0.39        25
           1       0.50      0.44      0.47        25
           2       0.20      0.21      0.21        14
           3       0.18      0.16      0.17        25
           4       0.73      0.58      0.64        33
           5       0.36      0.19      0.25        26
           6       0.26      0.21      0.23        29
           7       0.14      0.14      0.14        28
           8       0.38      0.43      0.40        21
           9       0.50      0.36      0.42        33
          10       0.00      0.00      0.00        23
          11       0.58      0.61      0.59        36
          12       0.84      1.00      0.91        21
          13       0.52      0.59      0.55        27
          14       0.41      